# 1) 순환 신경망
## 2. 케라스로 RNN 구현하기

In [7]:
from tensorflow.keras.layers import SimpleRNN

# 책에선 없지만 없으면 model 안됨
from tensorflow.keras.models import Sequential
model = Sequential()

model.add(SimpleRNN(hidden_units))

NameError: ignored

In [8]:
# 추가 인자를 사용할 때
model.add(SimpleRNN(hidden_units, input_shape=(timesteps, input_dim)))

# 다른 표기
model.add(SimpleRNN(hidden_units, input_length=M, input_dim=N))

NameError: ignored

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN

model = Sequential()
model.add(SimpleRNN(3, input_shape=(2,10)))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 3)                 42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


In [10]:
# batch_size 정의
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8, 2, 10)))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (8, 3)                    42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


## 3. 파이썬으로 RNN 구현하기

In [11]:
# 초기 은닉 상태 출력
import numpy as np

timesteps = 10
input_dim = 4
hidden_units = 8    # 은닉 상태의 크기를 8로 지정하였으므로 8의 차원을 가지는 0의 값으로 구성된 벡터 출력

# 입력에 해당되는 2D 텐서
inputs = np.random.random((timesteps, input_dim))

# 초기 은닉 상태는 0(벡터)로 초기화
hidden_state_t = np.zeros((hidden_units,))

print('초기 은닉 상태 :', hidden_state_t)

초기 은닉 상태 : [0. 0. 0. 0. 0. 0. 0. 0.]


In [12]:
# 가중치와 편향을 각 크기에 맞게 정의하고 크기를 출력
Wx = np.random.random((hidden_units, input_dim))    # (8, 4)크기의 2D 텐서 생성. 입력에 대한 가중치
Wh = np.random.random((hidden_units, hidden_units))    #(8, 8)크기의 2D 텐서 생성. 은닉 상태에 대한 가중치
b = np.random.random((hidden_units,))    # (8,)크기의 1D 텐서 생성. 이 값은 편향(bias)

print("가중치 Wx의 크기(shape) :", np.shape(Wx))
print("가중치 Wh의 크기(shape) :", np.shape(Wh))
print("편향의 크기(shape) :", np.shape(b))

가중치 Wx의 크기(shape) : (8, 4)
가중치 Wh의 크기(shape) : (8, 8)
편향의 크기(shape) : (8,)


In [13]:
total_hidden_states = []

# 각 시점 별 입력값
for input_t in inputs:

  # Wx * Xt + Wh * Ht-1 + b(bias)
  output_t = np.tanh(np.dot(Wx, input_t) + np.dot(Wh, hidden_state_t) + b)

  # 각 시점 t별 메모리 셀의 출력의 크기는 (timestep t, output_dim)
  # 각 시점의 은닉 상태의 값을 계속해서 누적
  total_hidden_states.append(list(output_t))
  hidden_state_t = output_t

# 출력 시 값을 깔끔하게 해주는 용도
total_hidden_states = np.stack(total_hidden_states, axis = 0)

# (timesteps, output_dim)
print("모든 시점의 은닉 상태 :")
print(total_hidden_states)

모든 시점의 은닉 상태 :
[[0.85892501 0.73176634 0.5251392  0.89855182 0.80492195 0.55550789
  0.95886989 0.94872285]
 [0.99958636 0.99737525 0.99843232 0.99994022 0.9995729  0.99958485
  0.99978895 0.99989431]
 [0.99993836 0.99965306 0.99990156 0.99999652 0.99998131 0.99993636
  0.99998939 0.99998785]
 [0.99979967 0.99896374 0.99961893 0.99997422 0.99950914 0.99984453
  0.99995005 0.99997152]
 [0.99979573 0.99878326 0.9993622  0.99997098 0.99938293 0.99987047
  0.99994438 0.99997303]
 [0.9998951  0.99919469 0.99971106 0.99998333 0.99977733 0.99990134
  0.99998262 0.99998682]
 [0.99995261 0.99930951 0.99983762 0.99998804 0.99991495 0.99993583
  0.99999148 0.99999357]
 [0.99989408 0.99958168 0.99983894 0.99999529 0.99996173 0.99991781
  0.99997969 0.99997959]
 [0.99991443 0.99957662 0.9998786  0.9999931  0.99993164 0.99990308
  0.99999216 0.99998852]
 [0.99981444 0.99908011 0.9994009  0.99998702 0.99982257 0.99990953
  0.99991842 0.99996495]]


## 4. 깊은 순환 신경망

In [14]:
# 은닉층을 2개 추가하는 경우의 코드
model = Sequential()
model.add(SimpleRNN(hidden_units, input_length=10, input_dim=5, return_sequences=True))
model.add(SimpleRNN(hidden_units, return_sequences=True))

## 5. 양방향 순환 신경망

In [16]:
# 앞 시점의 은닉상태와 뒤 시점의 은닉 상태의 값 모두가 현재 시점의 출력층에서 출력값을 예측하기 위해 사용
from tensorflow.keras.models import Bidirectional

timesteps = 10
input_dim = 5

model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_units, return_sequences=True), input_shape=(timesteps, input_dim)))

ImportError: ignored

# 4) 케라스의 SimpleRNN과 LSTM 이해하기
## 1. 임의의 입력 생성하기

In [18]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, LSTM, Bidirectional

In [20]:
# RNN과 LSTM을 테스트하기 위한 임의의 입력
train_X = [[0.1, 4.2, 1.5, 1.1, 2.8], [1.0, 3.1, 2.5, 0.7, 1.1], [0.3, 2.1, 1.5, 2.1, 0.1], [2.2, 1.4, 0.5, 0.9, 1.1]]
print(np.shape(train_X))

(4, 5)


In [21]:
# 2D 텐서를 3D 텐서로 변경
train_X = [[[0.1, 4.2, 1.5, 1.1, 2.8], [1.0, 3.1, 2.5, 0.7, 1.1], [0.3, 2.1, 1.5, 2.1, 0.1], [2.2, 1.4, 0.5, 0.9, 1.1]]]
train_X = np.array(train_X, dtype=np.float32)
print(train_X.shape)

(1, 4, 5)


## SimpleRNN 이해하기

In [22]:
# 은닉 상태의 크기3, 두 인자 값이 모두 False일 때의 출력값
rnn = SimpleRNN(3)
# rnn = SimpleRNN(3, return_sequences=False, return_state=False)와 동일
hidden_state = rnn(train_X)

print("hidden state : {}, shape : {}".format(hidden_state, hidden_state.shape))

hidden state : [[ 0.92126375 -0.9438475  -0.9928897 ]], shape : (1, 3)


In [23]:
# return_sequences를 True로 지정하여 모든 시점의 은닉 상태를 출력
rnn = SimpleRNN(3, return_sequences=True)
hidden_states = rnn(train_X)

print("hidden states : {}, shape : {}".format(hidden_states, hidden_states.shape))

hidden states : [[[ 0.9833128   0.9935881  -0.9993068 ]
  [ 0.85626465  0.99619716 -0.9940144 ]
  [-0.02891342  0.1666882  -0.925371  ]
  [-0.6240324   0.92511374 -0.93153566]]], shape : (1, 4, 3)


In [24]:
# return_state가 True일 경우에는 return_sequences의 True/False 여부와 상관없이 마지막 시점의 은닉 상태를 출력
rnn = SimpleRNN(3, return_sequences=True, return_state=True)
hidden_states, last_state = rnn(train_X)

print("hidden states : {}, shape : {}".format(hidden_states, hidden_states.shape))
print("last hidden state : {}, shape : {}".format(last_state, last_state.shape))

hidden states : [[[-0.21191558  0.9247593  -0.5584665 ]
  [-0.16361693  0.78625154 -0.96553063]
  [ 0.2464614   0.21260898 -0.97729856]
  [ 0.5551516   0.98431927 -0.49670163]]], shape : (1, 4, 3)
last hidden state : [[ 0.5551516   0.98431927 -0.49670163]], shape : (1, 3)


In [25]:
# return_sequences는 False, return_state는 True인 경우
# 두 개의 출력 모두 마지막 시점의 은닉 상태를 출력
rnn = SimpleRNN(3, return_sequences=False, return_state=True)
hidden_state, last_state = rnn(train_X)

print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))
print('last hidden state : {}, shape: {}'.format(last_state, last_state.shape))

hidden state : [[ 0.9712757  -0.9757286   0.04037877]], shape: (1, 3)
last hidden state : [[ 0.9712757  -0.9757286   0.04037877]], shape: (1, 3)


## LSTM 이해하기

In [26]:
# 임의의 입력에 대해서 LSTM을 사용할 경우
lstm = LSTM(3, return_sequences=False, return_state=True)
hidden_state, last_state, last_cell_state = lstm(train_X)

print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))
print('last hidden state : {}, shape: {}'.format(last_state, last_state.shape))
print('last cell state : {}, shape: {}'.format(last_cell_state, last_cell_state.shape))

hidden state : [[-0.12563483 -0.30545184  0.09709059]], shape: (1, 3)
last hidden state : [[-0.12563483 -0.30545184  0.09709059]], shape: (1, 3)
last cell state : [[-0.65726584 -1.1308584   0.24708945]], shape: (1, 3)


In [27]:
# LSTM은 return_state를 True로 둔 경우에는 마지막 시점의 은닉 상태뿐만 아니라 셀 상태까지 반환
# return_sequences = True
lstm = LSTM(3, return_sequences=True, return_state=True)
hidden_states, last_hidden_state, last_cell_state = lstm(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('last hidden state : {}, shape: {}'.format(last_hidden_state, last_hidden_state.shape))
print('last cell state : {}, shape: {}'.format(last_cell_state, last_cell_state.shape))

hidden states : [[[ 2.9361385e-01 -7.8314334e-02  6.8932626e-04]
  [ 3.1340003e-01 -4.3612391e-01 -3.3691153e-04]
  [ 4.2513573e-01 -5.2452689e-01  4.2103127e-02]
  [ 4.2195633e-01 -5.5606651e-01  4.5449071e-02]]], shape: (1, 4, 3)
last hidden state : [[ 0.42195633 -0.5560665   0.04544907]], shape: (1, 3)
last cell state : [[ 0.642616   -2.176991    0.06499276]], shape: (1, 3)


## 3. Bidirectional(LSTM) 이해하기

In [28]:
k_init = tf.keras.initializers.Constant(value=0.1)
b_init = tf.keras.initializers.Constant(value=0)
r_init = tf.keras.initializers.Constant(value=0.1)

In [29]:
# return_sequences가 False이고, return_state가 True인 경우
bilstm = Bidirectional(LSTM(3, return_sequences=False, return_state=True, \
                            kernel_initializer=k_init, bias_initializer=b_init, recurrent_initializer=r_init))
hidden_states, forward_h, forward_c, backward_h, backward_c = bilstm(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('forward state : {}, shape: {}'.format(forward_h, forward_h.shape))
print('backward state : {}, shape: {}'.format(backward_h, backward_h.shape))

hidden states : [[0.63031393 0.63031393 0.63031393 0.7038734  0.7038734  0.7038734 ]], shape: (1, 6)
forward state : [[0.63031393 0.63031393 0.63031393]], shape: (1, 3)
backward state : [[0.7038734 0.7038734 0.7038734]], shape: (1, 3)


In [30]:
# 현재 은닉 상태의 값을 고정시켜두었기 때문에 return_sequences를 True로 할 경우, 출력이 어떻게 바뀌는지 비교 가능
bilstm = Bidirectional(LSTM(3, return_sequences=True, return_state=True, \
                            kernel_initializer=k_init, bias_initializer=b_init, recurrent_initializer=r_init))
hidden_states, forward_h, forward_c, backward_h, backward_c = bilstm(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('forward state : {}, shape: {}'.format(forward_h, forward_h.shape))
print('backward state : {}, shape: {}'.format(backward_h, backward_h.shape))

hidden states : [[[0.35906473 0.35906473 0.35906473 0.7038734  0.7038734  0.7038734 ]
  [0.5511133  0.5511133  0.5511133  0.58863586 0.58863586 0.58863586]
  [0.59115744 0.59115744 0.59115744 0.3951699  0.3951699  0.3951699 ]
  [0.63031393 0.63031393 0.63031393 0.21942244 0.21942244 0.21942244]]], shape: (1, 4, 6)
forward state : [[0.63031393 0.63031393 0.63031393]], shape: (1, 3)
backward state : [[0.7038734 0.7038734 0.7038734]], shape: (1, 3)


# 6) RNN을 이용한 텍스트 생성
## 1. RNN을 이용하여 택스트 생성하기
### 1) 데이터에 대한 이해와 전처리

In [31]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [32]:
# 단어 집합을 생성하고 크기 확인
text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
vocab_size = len(tokenizer.word_index) + 1
print("단어 집합의 크기 : %d" % vocab_size)

단어 집합의 크기 : 12


In [33]:
# 각 단어와 단어에 부여된 정수 인덱스 출력
print(tokenizer.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [34]:
# 훈련 데이터 작성
sequences = list()
for line in text.split('\n'):    # 줄바꿈 문자를 기준으로 문장 토큰화
  encoded = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(encoded)):
    sequence = encoded[:i+1]
    sequences.append(sequence)

print("학습에 사용할 샘플의 개수 : %d" %len(sequences))

학습에 사용할 샘플의 개수 : 11


In [35]:
# 전체 샘플 출력
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [36]:
# 가장 긴 샘플을 기준으로 전체 샘플의 길이 일치시키기
max_len = max(len(l) for l in sequences)    # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print("샘플의 최대 길이 : {}".format(max_len))

샘플의 최대 길이 : 6


In [37]:
# 가장 긴 샘플의 길이로 패딩
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

In [38]:
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [39]:
# 각 샘플의 마지막 단어를 레이블로 분리
sequences = np.array(sequences)
x = sequences[:, :-1]     # 리스트의 마지막 값을 제외하고 저장
y = sequences[:, -1]    # 리스트의 마지막 값만 저장
print(x)
print(y)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]
[ 3  1  4  5  1  7  1  9 10  1 11]


In [40]:
# 레이블에 대해서 원-핫 인코딩 수행
y = to_categorical(y, num_classes = vocab_size)
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


### 2) 모델 설계하기

In [41]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [43]:
embedding_dim = 10
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 1s - loss: 2.4750 - accuracy: 0.0909 - 1s/epoch - 1s/step
Epoch 2/200
1/1 - 0s - loss: 2.4625 - accuracy: 0.0909 - 9ms/epoch - 9ms/step
Epoch 3/200
1/1 - 0s - loss: 2.4498 - accuracy: 0.0909 - 7ms/epoch - 7ms/step
Epoch 4/200
1/1 - 0s - loss: 2.4369 - accuracy: 0.0909 - 5ms/epoch - 5ms/step
Epoch 5/200
1/1 - 0s - loss: 2.4237 - accuracy: 0.0909 - 5ms/epoch - 5ms/step
Epoch 6/200
1/1 - 0s - loss: 2.4101 - accuracy: 0.1818 - 6ms/epoch - 6ms/step
Epoch 7/200
1/1 - 0s - loss: 2.3959 - accuracy: 0.0909 - 8ms/epoch - 8ms/step
Epoch 8/200
1/1 - 0s - loss: 2.3812 - accuracy: 0.1818 - 8ms/epoch - 8ms/step
Epoch 9/200
1/1 - 0s - loss: 2.3659 - accuracy: 0.3636 - 6ms/epoch - 6ms/step
Epoch 10/200
1/1 - 0s - loss: 2.3498 - accuracy: 0.3636 - 5ms/epoch - 5ms/step
Epoch 11/200
1/1 - 0s - loss: 2.3329 - accuracy: 0.4545 - 5ms/epoch - 5ms/step
Epoch 12/200
1/1 - 0s - loss: 2.3152 - accuracy: 0.4545 - 7ms/epoch - 7ms/step
Epoch 13/200
1/1 - 0s - loss: 2.2967 - accuracy: 0.4545 - 6ms/e

In [49]:
# 모델이 정확하게 예측하고 있는지 문장을 생성하는 함수
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        # 현재 단어에 대한 정수 인코딩과 패딩
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
        # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면 break
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [50]:
print(sentence_generation(model, tokenizer, "경마장에", 4))

경마장에 있는 말이 뛰고 있다


In [51]:
print(sentence_generation(model, tokenizer, '그의', 2))

그의 말이 법이다


In [52]:
print(sentence_generation(model, tokenizer, '가는', 5))

가는 말이 고와야 오는 말이 곱다
